In [16]:
import google.generativeai as genai

genai.configure(api_key="")

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro", safety_settings=safety_settings
)

In [20]:
import os
from flask import Flask, request, jsonify, render_template

# from google.cloud import translate_v2 as translate

app = Flask(__name__)

# Load Gemini API key from environment variable
GEMINI_API_KEY = ""

# Initialize Google Translate client
# translate_client = translate.Client()


def process_tell_us_about_you_essay(userInput):
    prompt = f"""
    Enhance this "Tell Us About You" scholarship essay. Focus on:
    1. Personal challenges and how they were overcome
    2. Responsibilities taken on within family or community
    3. Significant achievements
    4. Future aspirations and their connection to past experiences

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_how_are_you_unique_essay(userInput):
    prompt = f"""
    Enhance this "How Are You Unique?" scholarship essay. Focus on:
    1. Unique aspects of background, identity, interests, or talents
    2. Significance of these unique qualities
    3. Specific examples demonstrating uniqueness
    4. How these qualities contribute to academic or personal goals

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_failure_and_learning_essay(userInput):
    prompt = f"""
    Enhance this "Tell Us About a Time You Failed and What You Learned From It" essay. Focus on:
    1. Description of the failure
    2. Impact of the failure
    3. Emotions and needs that arose from the experience
    4. Actions taken to address the failure
    5. Lessons learned and how they've been applied

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_academic_and_career_goals_essay(userInput):
    prompt = f"""
    Enhance this "What Are Your Academic and Career Goals?" essay. Focus on:
    1. Specific academic goals
    2. Career aspirations
    3. Challenges that shaped these goals
    4. Steps taken or planned to achieve these goals
    5. How these goals align with personal values or experiences

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_community_contribution_essay(userInput):
    prompt = f"""
    Enhance this "How Have You Contributed to Your Community?" essay. Focus on:
    1. Specific community service project or contribution
    2. Problem or challenge addressed
    3. Actions taken and individual role
    4. Impact on the community
    5. Personal growth and lessons learned

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_deserve_scholarship_essay(userInput):
    prompt = f"""
    Enhance this "Why Do You Deserve This Scholarship?" essay. Focus on:
    1. Personal qualities that make you a strong candidate
    2. Academic or extracurricular achievements
    3. Challenges overcome
    4. Future potential and how the scholarship will help
    5. Alignment with the scholarship's mission or values

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_scholarship_help_essay(userInput):
    prompt = f"""
    Enhance this "How Will This Scholarship Help You?" essay. Focus on:
    1. Specific ways the scholarship will be used
    2. Financial challenges or needs
    3. Academic or career goals supported by the scholarship
    4. Potential impact on your future
    5. How you plan to pay it forward or contribute to society

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_sports_impact_essay(userInput):
    prompt = f"""
    Enhance this "What Impact Has Sports Had on Your Life?" essay. Focus on:
    1. Specific sport or athletic activity
    2. Life lessons learned through sports
    3. Personal growth and character development
    4. Challenges overcome through sports
    5. How sports have influenced other areas of life

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_why_study_pursue_essay(userInput):
    prompt = f"""
    Enhance this "Why Do You Want to Study/Pursue [X]?" essay. Focus on:
    1. Specific field of study or career path
    2. Experiences that sparked interest in this field
    3. Relevant skills or knowledge already acquired
    4. Future goals related to this field
    5. Potential impact or contribution to the field

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def process_belief_challenged_essay(userInput):
    prompt = f"""
    Enhance this "Tell Us About a Time When You Had a Belief or Idea Challenged" essay. Focus on:
    1. Initial belief or idea
    2. Situation or experience that challenged this belief
    3. Thought process and emotions during the challenge
    4. How the belief or idea changed (if it did)
    5. Lessons learned or personal growth from this experience

    Essay: {userInput}

    Enhanced essay:
    """
    response = model.generate_content(prompt)
    return clean_and_enhance_essay(response.text)


def detect_and_translate(text, target_language="en"):
    if isinstance(text, bytes):
        text = text.decode("utf-8")
    detection = translate_client.detect_language(text)
    if detection["language"] != target_language:
        result = translate_client.translate(text, target_language=target_language)
        return result["translatedText"]
    return text


def translate_to_english(text, target_language="en"):
    if isinstance(text, bytes):
        text = text.decode("utf-8")
    result = translate_client.translate(text, target_language=target_language)
    return result["translatedText"]


def clean_and_remove_fillers(userInput):
    cleanPrompt = f"""
    Clean up this text by removing filler words, stammering, and repetitions. Maintain the original meaning and key information.

    Input: {userInput}

    Output:
    """
    response = model.generate_content(cleanPrompt)
    return response.text


def refine_and_optimize_essay(essay, prompt_type):
    refinePrompt = f"""
    Refine and optimize this essay for a scholarship application. Tailor it to the "{prompt_type}" prompt while maintaining the original tone and voice. Ensure it doesn't sound artificial.

    Guidelines:
    1. Enhance the narrative structure
    2. Emphasize personal growth and challenges overcome
    3. Maintain authenticity and original voice
    4. Ensure relevance to the specific prompt
    5. Avoid clichés and overly formal language

    Essay: {essay}

    Refined essay:
    """
    response = model.generate_content(refinePrompt)
    return response.text


def format_essay(essay):
    formatPrompt = f"""
    Format this essay with proper paragraph structure and punctuation. Remove any markdown indicators and ensure it's in plain text format.

    Essay: {essay}

    Formatted essay:
    """
    response = model.generate_content(formatPrompt)
    return response.text


def process_essay(essay_type: str, user_input: str, needs_translation: bool) -> str:
    # Translate if needed and checkbox is checked
    if needs_translation:
        user_input = detect_and_translate(user_input)

    # Clean and remove fillers
    cleaned_essay = clean_and_remove_fillers(user_input)

    # Process based on essay type
    processing_functions = {
        "tell_us_about_you": process_tell_us_about_you_essay,
        "how_are_you_unique": process_how_are_you_unique_essay,
        "failure_and_learning": process_failure_and_learning_essay,
        "academic_and_career_goals": process_academic_and_career_goals_essay,
        "community_contribution": process_community_contribution_essay,
        "deserve_scholarship": process_deserve_scholarship_essay,
        "scholarship_help": process_scholarship_help_essay,
        "sports_impact": process_sports_impact_essay,
        "why_study_pursue": process_why_study_pursue_essay,
        "belief_challenged": process_belief_challenged_essay,
    }

    if essay_type in processing_functions:
        processed_essay = processing_functions[essay_type](cleaned_essay)
    else:
        return "Invalid essay type"

    # Refine and optimize
    refined_essay = refine_and_optimize_essay(processed_essay, essay_type)

    # Format
    final_essay = format_essay(refined_essay)

    return final_essay


@app.route("/", methods=["GET"])
def index():
    return render_template("index.html")


@app.route("/process", methods=["POST"])
def process():
    data = request.json
    essay_type = data["essay_type"]
    user_input = data["essay_content"]
    needs_translation = data["needs_translation"]

    final_essay = process_essay(essay_type, user_input, needs_translation)

    return jsonify({"processed_essay": final_essay})

ImportError: cannot import name 'translate_v2' from 'google.cloud' (unknown location)

In [18]:
process_essay(
    "tell_us_about_you",
    "I am a student at the University of California, studying computer science. I have always been passionate about technology and innovation. I am also involved in various extracurricular activities, such as coding clubs and hackathons. In my free time, I enjoy playing video games and reading science fiction novels.",
    needs_translation=False,
)

"From the soft whir of my grandfather's old computer, the soundtrack to my childhood, my passion for computer science emerged. Watching lines of code flicker across the screen, I felt not just curiosity, but a pull towards the limitless possibilities they represented. That fascination bloomed into a passion, leading me to pursue Computer Science at the University of California, a place where I could transform my love for coding into a future of meaningful innovation.\n\nLife, however, rarely follows a straight line of code. During my sophomore year of high school, my father lost his job, and our family's financial security vanished. Suddenly, my aspirations felt both urgent and impossibly distant. Determined to contribute, I started tutoring younger students in math and science, juggling long hours with my own studies. It wasn't easy, but it forged in me a relentless work ethic and taught me the importance of adapting to unexpected challenges – qualities that continue to serve me well 

In [19]:
if __name__ == "__main__":
    port = int(os.environ.get("PORT", 8080))
    app.run(debug=False, host="0.0.0.0", port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.143:8080
Press CTRL+C to quit
[2024-07-23 16:19:51,343] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^